In [18]:
# Dependecies
import requests
import json
import pandas as pd
import pymongo
from pymongo import MongoClient

In [15]:
# Mongo Connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [35]:
recipes = pd.read_csv("Resources/RAW_recipes.csv")
recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [27]:
from pprint import pprint
test_df = recipes.head(200)
json_df = test_df.to_json(orient='index')
pprint(json_df)

('{"0":{"name":"arriba   baked winter squash mexican '
 'style","id":137739,"minutes":55,"contributor_id":47892,"submitted":"2005-09-16","tags":"[\'60-minutes-or-less\', '
 "'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', "
 "'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', "
 "'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', "
 '\'christmas\', \'seasonal\', \'squash\']","nutrition":"[51.5, 0.0, 13.0, '
 '0.0, 2.0, 0.0, 4.0]","n_steps":11,"steps":"[\'make a choice and proceed with '
 "recipe', 'depending on size of squash , cut into half or fourths', 'remove "
 "seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut "
 "squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , "
 'drizzle melted honey , butter , grated piloncillo over each cut squash '
 "piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again "
 'depending on size , for 40 minutes up to an hour , un

In [17]:
interactions = pd.read_csv("Resources/RAW_interactions.csv")
interactions.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [29]:
recipes['id'].nunique()

231637

In [32]:
len(recipes)

231637